In [1]:
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [2]:
table = "DSS_SINIESTROS_AUTO_1A.tsv"
location = "../../data/raw"
sep = '\t'
encoding = 'latin1'
decimal = ','

## Selecting columns to use

In [3]:
usable_cols = ["NUM_SECU_POL",
    "MODELO_VEHICULO",
    "FEC_DENU_SINI",
    "FECHA_LIQUIDACION",
    "FECHA_RECHAZO"]

## Creating siniestros.feahter

In [4]:
df = pd.read_csv(f"{location}/{table}",
                     sep=sep,
                     encoding=encoding,
                     decimal=decimal,
                     usecols = usable_cols)

C:\Users\Santi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.shape

(6089218, 5)

In [6]:
df.dtypes

NUM_SECU_POL           int64
MODELO_VEHICULO      float64
FEC_DENU_SINI         object
FECHA_LIQUIDACION     object
FECHA_RECHAZO         object
dtype: object

In [7]:
def lookup(s):
    dates = {date:pd.to_datetime(date, format="%d/%m/%Y") for date in s.unique()}
    return s.map(dates)

for fecha in [col for col in df.columns if "FEC" in col]:
    df[fecha] = lookup(df[fecha])

In [8]:
df.dtypes

NUM_SECU_POL                  int64
MODELO_VEHICULO             float64
FEC_DENU_SINI        datetime64[ns]
FECHA_LIQUIDACION    datetime64[ns]
FECHA_RECHAZO        datetime64[ns]
dtype: object

In [9]:
data = {col: [df[col].min(),df[col].max()] for col in [col for col in df.columns if "FEC" in col]}
pd.DataFrame(data, index =['MIN', 'MAX'])

,FEC_DENU_SINI,FECHA_LIQUIDACION,FECHA_RECHAZO
MIN,1992-03-12,2010-01-21,1998-06-01
MAX,2019-11-30,2019-11-30,2019-11-29


In [10]:
for fecha in [col for col in df.columns if "FEC" in col]:
    df[fecha] = df[fecha].dt.strftime('%Y-%m')

In [11]:
df[df["FEC_DENU_SINI"] > '2018-10']["FEC_DENU_SINI"].value_counts()

2018-11    241111
2018-12    240796
2019-03    233461
2019-01    219508
2019-04    209349
2019-02    206786
2019-05    205100
2019-07    192057
2019-06    182083
2019-08    170889
2019-09    147088
2019-10    127500
2019-11     75885
Name: FEC_DENU_SINI, dtype: int64

In [12]:
df[df["FEC_DENU_SINI"] > '2018-10'].reset_index(drop=True).to_feather("siniestros.feather")